In [4]:
import pandas as pd
import numpy as np
import csv
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
